# Momotaro RAG Experiment - MVP (LM Studio版)

このノートブックでは、桃太郎の物語に関するRAG（Retrieval-Augmented Generation）システムの最小機能を実装します。

## 目標
- 日本語の桃太郎テキストを読み込む
- テキストをチャンクに分割
- FAISSベクトルインデックスを作成
- テスト質問に対してRAG有無での回答を比較

## 必要な準備
1. `data/raw/japanese/` に桃太郎のテキストファイルを3-5個配置
2. **LM Studioを起動してローカルサーバーを開始**
   - LM Studioで Phi-3.5-mini-instruct モデルを読み込む
   - 「Local Server」タブでサーバーを起動（デフォルト: http://localhost:1234）
3. Ollamaが起動していることを確認（エンベディング用）
   - `ollama pull nomic-embed-text` を実行

## 1. セットアップとインポート

In [ ]:
import os
import sys
import json
from pathlib import Path
from typing import List, Dict, Tuple
import re

# Data processing
import numpy as np
import pandas as pd
from dotenv import load_dotenv

# Vector store
import faiss

# LLM - OpenAI-compatible API
from openai import OpenAI

# Embedding - Ollama
import ollama

# Encoding detection
from charset_normalizer import from_path

# Load environment variables
load_dotenv()

# Configuration
LLM_PROVIDER = os.getenv('LLM_PROVIDER', 'lmstudio')
LMSTUDIO_BASE_URL = os.getenv('LMSTUDIO_BASE_URL', 'http://localhost:1234/v1')
LMSTUDIO_MODEL_NAME = os.getenv('LMSTUDIO_MODEL_NAME', 'phi-3.5-mini-instruct')
EMBEDDING_MODEL = os.getenv('EMBEDDING_MODEL_NAME', 'nomic-embed-text')
CHUNK_SIZE = int(os.getenv('CHUNK_SIZE', 500))
CHUNK_OVERLAP = int(os.getenv('CHUNK_OVERLAP', 50))
TOP_K = int(os.getenv('TOP_K_RESULTS', 3))
VECTOR_DIM = int(os.getenv('VECTOR_DIMENSION', 768))

# Initialize LM Studio client (OpenAI-compatible)
lm_client = OpenAI(
    base_url=LMSTUDIO_BASE_URL,
    api_key="lm-studio"  # LM Studio doesn't require a real API key
)

# Experiment Configuration
EXPERIMENT_ID = "exp_002_baseline_lmstudio_phi35_nomic-v1"

# Paths - 実験単位で管理
PROJECT_ROOT = Path.cwd().parent.parent  # From experiments/exp_XXX/ to project root
EXPERIMENT_PATH = PROJECT_ROOT / "experiments" / EXPERIMENT_ID
DATA_RAW_PATH = PROJECT_ROOT / "data" / "raw" / "japanese"
DATA_VECTORS_PATH = EXPERIMENT_PATH / "vectors"
RESULTS_PATH = EXPERIMENT_PATH / "results"
CONFIG_PATH = EXPERIMENT_PATH / "config.json"

# Create paths if they don't exist
DATA_VECTORS_PATH.mkdir(parents=True, exist_ok=True)
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

print("✓ Setup complete")
print(f"  - Experiment ID: {EXPERIMENT_ID}")
print(f"  - LLM Provider: {LLM_PROVIDER}")
print(f"  - LM Studio URL: {LMSTUDIO_BASE_URL}")
print(f"  - LM Studio Model: {LMSTUDIO_MODEL_NAME}")
print(f"  - Embedding Model: {EMBEDDING_MODEL} (via Ollama)")
print(f"  - Data path: {DATA_RAW_PATH}")
print(f"  - Vectors path: {DATA_VECTORS_PATH}")
print(f"  - Results path: {RESULTS_PATH}")
print(f"\n⚠️  LM Studioが起動していることを確認してください！")

## 2. LM Studio接続テスト

In [3]:
# LM Studioサーバーへの接続をテスト
print("Testing LM Studio connection...")
try:
    # 利用可能なモデルのリストを取得
    models = lm_client.models.list()
    print("✓ LM Studio connection successful!")
    print(f"\nAvailable models:")
    for model in models.data:
        print(f"  - {model.id}")
    
    # 簡単なテスト生成
    print("\nTesting generation...")
    test_response = lm_client.chat.completions.create(
        model=LMSTUDIO_MODEL_NAME,
        messages=[
            {"role": "user", "content": "こんにちは！"}
        ],
        temperature=0.7,
        max_tokens=50
    )
    print(f"Test response: {test_response.choices[0].message.content}")
    print("\n✓ LM Studio is working correctly!")
    
except Exception as e:
    print(f"\n❌ Error connecting to LM Studio: {e}")
    print("\nPlease ensure:")
    print("1. LM Studio is running")
    print("2. A model is loaded in LM Studio")
    print("3. The local server is started (usually http://localhost:1234)")
    print(f"4. The base URL in .env matches: {LMSTUDIO_BASE_URL}")

Testing LM Studio connection...
✓ LM Studio connection successful!

Available models:
  - phi-3.5-mini-instruct
  - phi-3.5-mini-instruct:2
  - phi-3-mini-4k-instruct
  - qwen2.5-7b-instruct-1m
  - gemma-2-9b-it
  - text-embedding-nomic-embed-text-v1.5

Testing generation...
Test response: こんにちは！今日も素敵な一コマでお手伝いします。何を考えているので、どうぞ教えてくださ

✓ LM Studio is working correctly!


## 3. データロードと前処理

In [4]:
def detect_encoding(file_path: Path) -> str:
    """
    ファイルのエンコーディングを自動検出
    """
    result = from_path(file_path).best()
    return result.encoding if result else 'utf-8'

def load_text_file(file_path: Path) -> Dict[str, str]:
    """
    テキストファイルを読み込み、メタデータとともに返す
    """
    encoding = detect_encoding(file_path)
    
    try:
        with open(file_path, 'r', encoding=encoding) as f:
            content = f.read()
        
        return {
            'source': file_path.name,
            'path': str(file_path),
            'language': 'ja',
            'encoding': encoding,
            'content': content,
            'char_count': len(content)
        }
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def preprocess_text(text: str) -> str:
    """
    テキストの前処理
    - 余分な空白を削除
    - 改行を正規化
    """
    # 連続する空白を1つに
    text = re.sub(r'[ \t]+', ' ', text)
    
    # 3つ以上の連続する改行を2つに
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # 行頭・行末の空白を削除
    text = '\n'.join(line.strip() for line in text.split('\n'))
    
    return text.strip()

# データの読み込み
print("Loading Japanese Momotaro texts...")
text_files = list(DATA_RAW_PATH.glob('*.txt'))

if not text_files:
    print(f"⚠ No text files found in {DATA_RAW_PATH}")
    print("Please add Momotaro text files to data/raw/japanese/")
    documents = []
else:
    documents = []
    for file_path in text_files:
        doc = load_text_file(file_path)
        if doc:
            doc['content'] = preprocess_text(doc['content'])
            documents.append(doc)
            print(f"  ✓ Loaded: {doc['source']} ({doc['char_count']:,} chars, {doc['encoding']})")
    
    print(f"\n✓ Total documents loaded: {len(documents)}")

Loading Japanese Momotaro texts...
  ✓ Loaded: aozora_jp_001.txt (2,001 chars, utf_8)
  ✓ Loaded: children_jp_003.txt (1,454 chars, utf_8)
  ✓ Loaded: traditional_jp_002.txt (3,197 chars, utf_8)

✓ Total documents loaded: 3


## 4. テキストのチャンク化

In [5]:
def chunk_by_paragraphs(text: str, max_chars: int = CHUNK_SIZE, overlap: int = CHUNK_OVERLAP) -> List[str]:
    """
    段落ベースでテキストをチャンク化
    段落が長すぎる場合は文単位で分割
    """
    # 段落で分割（空行で区切られた部分）
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    
    chunks = []
    current_chunk = ""
    
    for para in paragraphs:
        # 段落が最大サイズを超える場合
        if len(para) > max_chars:
            # 現在のチャンクを保存
            if current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = ""
            
            # 長い段落を文単位で分割
            sentences = re.split(r'([。！？])', para)
            sentences = [''.join(sentences[i:i+2]) for i in range(0, len(sentences), 2)]
            
            for sent in sentences:
                if len(current_chunk) + len(sent) <= max_chars:
                    current_chunk += sent
                else:
                    if current_chunk:
                        chunks.append(current_chunk.strip())
                    current_chunk = sent
        else:
            # 段落を追加できるか確認
            if len(current_chunk) + len(para) + 2 <= max_chars:  # +2 for \n\n
                if current_chunk:
                    current_chunk += "\n\n" + para
                else:
                    current_chunk = para
            else:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = para
    
    # 最後のチャンクを追加
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

def create_chunks_with_metadata(documents: List[Dict]) -> List[Dict]:
    """
    全ドキュメントをチャンク化し、メタデータを付加
    """
    all_chunks = []
    
    for doc in documents:
        chunks = chunk_by_paragraphs(doc['content'], CHUNK_SIZE, CHUNK_OVERLAP)
        
        for idx, chunk_text in enumerate(chunks):
            chunk = {
                'chunk_id': f"{doc['source']}_{idx:03d}",
                'text': chunk_text,
                'source': doc['source'],
                'language': doc['language'],
                'position': idx,
                'char_count': len(chunk_text)
            }
            all_chunks.append(chunk)
    
    return all_chunks

# チャンク作成
if documents:
    print("Creating chunks...")
    chunks = create_chunks_with_metadata(documents)
    print(f"✓ Created {len(chunks)} chunks")
    
    # チャンクのサンプル表示
    if chunks:
        print("\nSample chunks:")
        for i, chunk in enumerate(chunks[:3]):
            print(f"\n--- Chunk {i+1}: {chunk['chunk_id']} ---")
            print(f"Source: {chunk['source']}")
            print(f"Length: {chunk['char_count']} chars")
            print(f"Text preview: {chunk['text'][:100]}...")
else:
    chunks = []
    print("⚠️  No documents to chunk. Please add text files first.")

Creating chunks...
✓ Created 15 chunks

Sample chunks:

--- Chunk 1: aozora_jp_001.txt_000 ---
Source: aozora_jp_001.txt
Length: 493 chars
Text preview: 桃太郎

むかしむかし、あるところに、おじいさんとおばあさんが住んでいました。

一、桃太郎の誕生

おじいさんは山へ柴刈りに、おばあさんは川へ洗濯に行きました。おばあさんが川で洗濯をしていると、どん...

--- Chunk 2: aozora_jp_001.txt_001 ---
Source: aozora_jp_001.txt
Length: 462 chars
Text preview: ある日、桃太郎はおじいさんとおばあさんに言いました。

「鬼ヶ島へ行って、鬼を退治してきます。村のみんなのためにも、育ててくれたお二人のためにも、僕が鬼をやっつけます」

おばあさんは心配しましたが、...

--- Chunk 3: aozora_jp_001.txt_002 ---
Source: aozora_jp_001.txt
Length: 434 chars
Text preview: 桃太郎は猿にもきびだんごを一つあげました。猿も喜んで、桃太郎のお供になりました。

さらに進んでいくと、今度は一羽のキジに出会いました。

「桃太郎さん、どこへ行かれるのですか」
「鬼ヶ島へ、鬼退治に...


## 5. エンベディング生成とFAISSインデックス構築

エンベディングはOllamaのnomic-embed-textを使用します。

In [6]:
def generate_embedding(text: str, model: str = EMBEDDING_MODEL) -> np.ndarray:
    """
    Ollamaを使ってテキストのエンベディングを生成
    """
    try:
        response = ollama.embeddings(model=model, prompt=text)
        return np.array(response['embedding'], dtype=np.float32)
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return None

def generate_embeddings_batch(texts: List[str], model: str = EMBEDDING_MODEL) -> np.ndarray:
    """
    複数テキストのエンベディングを生成
    """
    embeddings = []
    
    for i, text in enumerate(texts):
        if (i + 1) % 10 == 0:
            print(f"  Processing {i+1}/{len(texts)}...")
        
        emb = generate_embedding(text, model)
        if emb is not None:
            embeddings.append(emb)
        else:
            # エラーの場合はゼロベクトル
            embeddings.append(np.zeros(VECTOR_DIM, dtype=np.float32))
    
    return np.array(embeddings)

def create_faiss_index(embeddings: np.ndarray) -> faiss.Index:
    """
    FAISSインデックスを作成
    """
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # L2距離を使用
    index.add(embeddings)
    return index

# エンベディング生成
if chunks:
    print("Generating embeddings...")
    print("This may take a few minutes depending on the number of chunks.")
    print("⚠️  Ollamaが起動していることを確認してください！")
    
    chunk_texts = [chunk['text'] for chunk in chunks]
    embeddings = generate_embeddings_batch(chunk_texts)
    
    print(f"✓ Generated {len(embeddings)} embeddings")
    print(f"  Embedding dimension: {embeddings.shape[1]}")
    
    # FAISSインデックス作成
    print("\nCreating FAISS index...")
    index = create_faiss_index(embeddings)
    print(f"✓ FAISS index created with {index.ntotal} vectors")
    
    # インデックスとメタデータを保存
    index_path = DATA_VECTORS_PATH / "index.faiss"
    metadata_path = DATA_VECTORS_PATH / "metadata.json"
    
    faiss.write_index(index, str(index_path))
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)
    
    print(f"\n✓ Saved index to {index_path}")
    print(f"✓ Saved metadata to {metadata_path}")
else:
    print("⚠️  No chunks to embed. Please add text files and create chunks first.")

Generating embeddings...
This may take a few minutes depending on the number of chunks.
⚠️  Ollamaが起動していることを確認してください！
  Processing 10/15...
✓ Generated 15 embeddings
  Embedding dimension: 768

Creating FAISS index...
✓ FAISS index created with 15 vectors

✓ Saved index to /Users/igomuni/MyGitHub/RagLab001/data/vectors/index.faiss
✓ Saved metadata to /Users/igomuni/MyGitHub/RagLab001/data/vectors/metadata.json


## 6. 検索機能の実装

In [7]:
def search_similar_chunks(query: str, index: faiss.Index, chunks: List[Dict], k: int = TOP_K) -> List[Dict]:
    """
    クエリに類似したチャンクを検索
    """
    # クエリのエンベディング生成
    query_embedding = generate_embedding(query)
    if query_embedding is None:
        return []
    
    # 検索実行
    query_embedding = query_embedding.reshape(1, -1)
    distances, indices = index.search(query_embedding, k)
    
    # 結果を整形
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        if idx < len(chunks):
            result = chunks[idx].copy()
            result['distance'] = float(dist)
            result['similarity'] = 1 / (1 + dist)  # 類似度スコア
            results.append(result)
    
    return results

# 検索のテスト
if chunks and 'index' in locals():
    print("Testing retrieval...")
    test_query = "桃太郎はどこから生まれましたか"
    results = search_similar_chunks(test_query, index, chunks, k=3)
    
    print(f"\nQuery: {test_query}")
    print(f"Found {len(results)} relevant chunks:\n")
    
    for i, result in enumerate(results, 1):
        print(f"Result {i}:")
        print(f"  Source: {result['source']}")
        print(f"  Chunk ID: {result['chunk_id']}")
        print(f"  Similarity: {result['similarity']:.4f}")
        print(f"  Text: {result['text'][:150]}...")
        print()
else:
    print("⚠️  Please create chunks and index first.")

Testing retrieval...

Query: 桃太郎はどこから生まれましたか
Found 3 relevant chunks:

Result 1:
  Source: aozora_jp_001.txt
  Chunk ID: aozora_jp_001.txt_000
  Similarity: 0.0067
  Text: 桃太郎

むかしむかし、あるところに、おじいさんとおばあさんが住んでいました。

一、桃太郎の誕生

おじいさんは山へ柴刈りに、おばあさんは川へ洗濯に行きました。おばあさんが川で洗濯をしていると、どんぶらこ、どんぶらこと、大きな桃が流れてきました。

「これは良い桃じゃ。持って帰っておじいさんと食べ...

Result 2:
  Source: aozora_jp_001.txt
  Chunk ID: aozora_jp_001.txt_002
  Similarity: 0.0064
  Text: 桃太郎は猿にもきびだんごを一つあげました。猿も喜んで、桃太郎のお供になりました。

さらに進んでいくと、今度は一羽のキジに出会いました。

「桃太郎さん、どこへ行かれるのですか」
「鬼ヶ島へ、鬼退治に行くのだ」
「それでは、腰につけた物を一つください。お供します」

桃太郎はキジにもきびだんごを一つ...

Result 3:
  Source: traditional_jp_002.txt
  Chunk ID: traditional_jp_002.txt_006
  Similarity: 0.0063
  Text: 「桃太郎さまが帰ってきた！」
「鬼を退治してくれた！」

人々は歓声を上げ、桃太郎たちを英雄として迎えました。

おじいさんとおばあさんは、無事に帰ってきた桃太郎を抱きしめて、喜びの涙を流しました。

桃太郎が持ち帰った宝物は村人たちに分け与えられ、村は豊かになりました。鬼たちは約束を守り、二度と村...



## 7. LLM統合（LM Studio + Phi3.5）

In [8]:
def generate_response_lmstudio(
    prompt: str, 
    model: str = LMSTUDIO_MODEL_NAME,
    temperature: float = 0.7, 
    max_tokens: int = 500
) -> str:
    """
    LM Studioを使って応答を生成
    """
    try:
        response = lm_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating response: {e}")
        return ""

def build_rag_prompt(query: str, context_chunks: List[Dict]) -> str:
    """
    RAG用のプロンプトを構築
    """
    context_text = "\n\n".join([
        f"[参考{i+1}] {chunk['text']}"
        for i, chunk in enumerate(context_chunks)
    ])
    
    prompt = f"""以下の参考情報を使って質問に答えてください。

【参考情報】
{context_text}

【質問】
{query}

【回答】
"""
    return prompt

def build_no_rag_prompt(query: str) -> str:
    """
    RAG無しのプロンプトを構築
    """
    prompt = f"""以下の質問に答えてください。

【質問】
{query}

【回答】
"""
    return prompt

# LLMのテスト
if chunks and 'index' in locals():
    print("Testing LLM generation with LM Studio...\n")
    
    test_query = "桃太郎はどこから生まれましたか"
    
    # RAGありの応答
    print("=== RAG Response ===")
    context = search_similar_chunks(test_query, index, chunks, k=3)
    rag_prompt = build_rag_prompt(test_query, context)
    rag_response = generate_response_lmstudio(rag_prompt)
    print(rag_response)
    
    print("\n" + "="*50 + "\n")
    
    # RAG無しの応答
    print("=== Non-RAG Response ===")
    no_rag_prompt = build_no_rag_prompt(test_query)
    no_rag_response = generate_response_lmstudio(no_rag_prompt)
    print(no_rag_response)
else:
    print("⚠️  Please create chunks and index first.")

Testing LLM generation with LM Studio...

=== RAG Response ===
桃太郎は、お母さんが川で大きな桃を持ち帰って家に持ち帰ったとき、その中から生まれました。この出来事は「一」部分で説明されているように、おじいさんや犬、猿など仲間が見守り、桃太郎を育て上げたと記載されています。


=== Non-RAG Response ===
提供された情報では、「桃太郎がどこから生まれたのか」について明確な詳細を示していないため、正確な回答を提供することはできません。桃太郎がどの国や都市で生まれたかについての情報がある場合は教えてください。そうしたら、適切な回答を提供することが可能です。

例えば：「桃太郎は日本の東京都に生まれました」と情報があれば、次のように回答することができます：

【回答】
桃太郎は日本の東京都から生まれました。

それ以外の場合は無謝に思います。


## 8. 評価実験：複数の質問で比較

In [9]:
# テスト質問セット
test_questions = [
    {
        'id': 'q1',
        'category': 'plot',
        'question': '桃太郎はどこから生まれましたか？'
    },
    {
        'id': 'q2',
        'category': 'plot',
        'question': '桃太郎は誰と一緒に鬼ヶ島へ行きましたか？'
    },
    {
        'id': 'q3',
        'category': 'detail',
        'question': '桃太郎は動物たちに何をあげましたか？'
    },
    {
        'id': 'q4',
        'category': 'plot',
        'question': '桃太郎は鬼ヶ島で何をしましたか？'
    },
    {
        'id': 'q5',
        'category': 'theme',
        'question': '桃太郎の物語が教える教訓は何ですか？'
    }
]

print(f"Running evaluation with {len(test_questions)} questions...\n")

Running evaluation with 5 questions...



In [10]:
from datetime import datetime

def run_comparison_experiment(questions: List[Dict], index: faiss.Index, chunks: List[Dict]) -> List[Dict]:
    """
    全ての質問に対してRAG有無の比較実験を実行
    """
    results = []
    
    for i, q in enumerate(questions, 1):
        print(f"\n{'='*60}")
        print(f"Question {i}/{len(questions)}: {q['question']}")
        print(f"{'='*60}")
        
        # RAGありの応答
        print("\nGenerating RAG response...")
        context = search_similar_chunks(q['question'], index, chunks, k=TOP_K)
        rag_prompt = build_rag_prompt(q['question'], context)
        rag_response = generate_response_lmstudio(rag_prompt)
        
        print("RAG Response:")
        print(rag_response)
        
        # RAG無しの応答
        print("\nGenerating Non-RAG response...")
        no_rag_prompt = build_no_rag_prompt(q['question'])
        no_rag_response = generate_response_lmstudio(no_rag_prompt)
        
        print("Non-RAG Response:")
        print(no_rag_response)
        
        # 結果を保存
        result = {
            'question_id': q['id'],
            'category': q['category'],
            'question': q['question'],
            'timestamp': datetime.now().isoformat(),
            'llm_provider': 'lmstudio',
            'model': LMSTUDIO_MODEL_NAME,
            'rag': {
                'response': rag_response,
                'context_chunks': [{
                    'chunk_id': c['chunk_id'],
                    'source': c['source'],
                    'text': c['text'],
                    'similarity': c['similarity']
                } for c in context],
                'response_length': len(rag_response)
            },
            'no_rag': {
                'response': no_rag_response,
                'response_length': len(no_rag_response)
            }
        }
        
        results.append(result)
    
    return results

# 実験実行
if chunks and 'index' in locals():
    experiment_results = run_comparison_experiment(test_questions, index, chunks)
else:
    print("⚠️  Please create chunks and index first.")
    experiment_results = []


Question 1/5: 桃太郎はどこから生まれましたか？

Generating RAG response...
RAG Response:
桃太郎は大きな桃から生まれました。おばあさんが川で洗濯をしていると、そこから流れてきたのです。中から元気に育った赤ちゃんが「桃太郎」と名付けられました。

Generating Non-RAG response...
Non-RAG Response:
提供された情報では、桃太郎の出身地について明確な詳細が記載されていないため、正しい回答を具体的に提供することはできません。桃太郎の出身地について知られている情報や文脈が必要です。

例えば、以下のような情報があれば：

【回答】
桃太郎は広島県出身です。

これに基づいて、その情報を提供した場合と考えられます。

Question 2/5: 桃太郎は誰と一緒に鬼ヶ島へ行きましたか？

Generating RAG response...
RAG Response:
桃太郎はキジ、犬、そして猿と共に鬼ヶ島へ向かいました。

参考情報3でおばあさんが日本一のきびだんごを作って桃太郎に与えたことや、キジと再度出会う場面（参考1と2）からも明らかなように、三匹は共に鬼ヶ島へ向かいました。

答えを推論するのであれば：「キジ」が最初に出会った家来（参考1）と再度出会うことや、二番目に出会った家来も同じく猿だから結論づけているのです。

したがって、桃太郎はキジ、犬、そして猿と共に鬼ヶ島へ向かいました。

Generating Non-RAG response...
Non-RAG Response:
桃太郎が鬼ヶ島への旅を共有していた人物は、弥三石と言われています。この話題に関連する資料や文学作品（例えば能「滑走路」で紹介されている）を参照して、弥三石という人物が正しい回答だったのかどうかを判断する必要があります。

例えば：

【回答】「桃太郎は鬼ヶ島へ行く際、弥三石と共に出発していた」 これらの情報源や文学作品から得られる事実です。

注意：正確な回答は質問が基ざったもので、その話題に関連している信頼性のある情報源を参照する必要があります。

注意：「桃太郎と弥三石」は日本古来の伝説で有名なキャラクターで、能や物語に出てくることも多いためそれらを参照して回答するべきです。

注

## 9. 結果の保存と分析

In [11]:
# 結果をJSONファイルに保存
if experiment_results:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = RESULTS_PATH / f"experiment_results_lmstudio_{timestamp}.json"
    
    with open(results_file, 'w', encoding='utf-8') as f:
        json.dump(experiment_results, f, ensure_ascii=False, indent=2)
    
    print(f"\n✓ Results saved to {results_file}")
else:
    print("⚠️  No results to save.")


✓ Results saved to /Users/igomuni/MyGitHub/RagLab001/results/comparisons/experiment_results_lmstudio_20260112_090749.json


In [12]:
# 結果の統計分析
if experiment_results:
    print("\n" + "="*60)
    print("EXPERIMENT SUMMARY")
    print("="*60 + "\n")
    
    rag_lengths = [r['rag']['response_length'] for r in experiment_results]
    no_rag_lengths = [r['no_rag']['response_length'] for r in experiment_results]
    
    print(f"Total questions: {len(experiment_results)}")
    print(f"LLM: {LMSTUDIO_MODEL_NAME} (via LM Studio)")
    print(f"\nResponse lengths:")
    print(f"  RAG average: {np.mean(rag_lengths):.1f} chars")
    print(f"  Non-RAG average: {np.mean(no_rag_lengths):.1f} chars")
    print(f"\nRAG used context from:")
    
    # 使用されたソースの集計
    sources_used = {}
    for r in experiment_results:
        for chunk in r['rag']['context_chunks']:
            source = chunk['source']
            sources_used[source] = sources_used.get(source, 0) + 1
    
    for source, count in sorted(sources_used.items(), key=lambda x: x[1], reverse=True):
        print(f"  {source}: {count} times")
    
    print("\n" + "="*60)
    print("\nManual evaluation template:")
    print("For each question, rate both responses on:")
    print("  1. Accuracy (1-5): Is the answer factually correct?")
    print("  2. Completeness (1-5): Does it cover all aspects?")
    print("  3. Coherence (1-5): Is it well-structured?")
    print("  4. Relevance (1-5): Does it answer the question?")
    print("\nResults are saved in JSON format for manual annotation.")
else:
    print("⚠️  No results to analyze.")


EXPERIMENT SUMMARY

Total questions: 5
LLM: Phi-3.5-mini-instruct (via LM Studio)

Response lengths:
  RAG average: 200.4 chars
  Non-RAG average: 276.8 chars

RAG used context from:
  aozora_jp_001.txt: 10 times
  traditional_jp_002.txt: 5 times


Manual evaluation template:
For each question, rate both responses on:
  1. Accuracy (1-5): Is the answer factually correct?
  2. Completeness (1-5): Does it cover all aspects?
  3. Coherence (1-5): Is it well-structured?
  4. Relevance (1-5): Does it answer the question?

Results are saved in JSON format for manual annotation.


## 10. 結果の可視化（オプション）

✓ Visualization complete


In [13]:
if experiment_results:
    import matplotlib.pyplot as plt
    import matplotlib
    
    # 日本語フォントの設定
    matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'Hiragino Sans', 'Yu Gothic', 'Meiryo']
    matplotlib.rcParams['axes.unicode_minus'] = False
    
    # 応答長の比較
    fig, ax = plt.subplots(figsize=(10, 6))
    
    question_ids = [r['question_id'] for r in experiment_results]
    x = np.arange(len(question_ids))
    width = 0.35
    
    ax.bar(x - width/2, rag_lengths, width, label='RAG', alpha=0.8)
    ax.bar(x + width/2, no_rag_lengths, width, label='Non-RAG', alpha=0.8)
    
    ax.set_xlabel('Question ID')
    ax.set_ylabel('Response Length (characters)')
    ax.set_title(f'Response Length Comparison: RAG vs Non-RAG\n(LM Studio: {LMSTUDIO_MODEL_NAME})')
    ax.set_xticks(x)
    ax.set_xticklabels(question_ids)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n✓ Visualization complete")
else:
    print("⚠️  No results to visualize.")

## 11. 次のステップ

このMVPが成功したら、以下の拡張を検討:

1. **多言語対応**: 英語・中国語のテキストを追加
2. **評価の自動化**: 自動評価メトリクスの実装
3. **チャンク戦略の最適化**: 異なるチャンクサイズやオーバーラップの実験
4. **リランキング**: 検索結果の再順位付け
5. **モジュール化**: コードをsrc/配下のモジュールに分割
6. **スクリプト化**: scripts/build_index.py と scripts/run_experiment.py の作成
7. **他のLLMとの比較**: Ollama版とLM Studio版の結果比較

---

**実験完了！**  
結果を確認して、RAGの効果を評価してください。